Install requirements

In [1]:
!pip install fastcore >/dev/null 2>&1
!pip install wandb >/dev/null 2>&1
!pip install ghapi >/dev/null 2>&1

Wandb login

In [2]:
import wandb
wandb.login()

wandb: Currently logged in as: eolecvka. Use `wandb login --relogin` to force relogin


True

Fetch sweep output data

In [6]:
import wandb,os
from fastcore.all import *
import pandas as pd

api = wandb.Api()

sweep_ids = [
    'eolecvka/fastai-img-models/sweeps/a6bqgnpb'
]
sweeps = concat(api.sweep(o).runs for o in sweep_ids)
summs = [{**r.summary, 'model_name':r.config['model_name']} for r in sweeps]

df = pd.DataFrame(summs)

#df['dataset'] = 'planet'
df['dataset'] = 'pets'
#df.loc[df['accuracy_multi'].isna(), 'dataset'] = 'pets'
#df.loc[df['dataset']=='planet', 'accuracy'] = df.loc[df['dataset']=='planet', 'accuracy_multi']
df['error_rate'] = 1-df.accuracy

pd.set_option('display.max_columns', None)
# df

Save as Gist csv file

In [7]:
from io import StringIO
import ghapi.core as gh

In [8]:
strm = StringIO()
cols = ['dataset', 'model_name', 'GPU_mem', 'error_rate', 'valid_loss', 'train_loss', 'fit_time']
df[cols].to_csv(strm, index=False)
txt = strm.getvalue()

In [11]:
# Assumes I have set the environment variable GITHUB_TOKEN prior to running this notebook
g = gh.GhApi(owner='eolecvk', repo='fastai-img-models', token=os.environ['GITHUB_TOKEN'])
gist = g.create_gist('fastai_timm repro', txt, filename='fastai_timm_repro.csv', public=True)

In [13]:
gist.html_url

'https://gist.github.com/5fb35bcd2536e8a78492da6a04686ddf'